In [1]:
import os,gc
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import cudf

In [2]:
df = cudf.read_parquet('../../data/infer_data/train.parquet')
df.ts = (df.ts//1000).astype('int32')

print( df.shape )
MN = df.ts.min()
df = df.loc[df.ts >= MN + 60*60*24*7].reset_index(drop=True)

print( df.shape )
df.head()

(216716096, 4)
(163555218, 4)


,session,aid,ts,type
0,0,1629651,1659964140,0
1,0,1768568,1660015113,0
2,0,1318324,1660015128,0
3,0,1840418,1660024261,0
4,0,1813509,1660024355,0


In [3]:
mx = df.ts.max()
df = df.loc[df.ts > mx - 60*60*24*21]

In [4]:
mn = df.ts.min()
#df['hour'] = df.ts % (60*60*24)
df['day'] = (df.ts-mn)  // (60*60*24) #% (60*60*24*7)

In [5]:
AIDS = df.aid.unique().values
DAYS = df.day.unique().values

In [6]:
DAYS

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20])

In [7]:
tmp = df.groupby(['aid','day']).agg({'aid':'count','session':'count'})
tmp.columns = ['item','user']

In [8]:
tmp.head()

,,item,user
aid,day,,
1032706,5,8,8
247381,18,1,1
653273,18,3,3
11948,13,1,1
649741,6,7,7


In [9]:
import cupy
dff = cudf.DataFrame(columns=['aid','day'])
dff['aid'] = cupy.repeat(AIDS,21).astype('int32')
dff['day'] = cupy.tile(DAYS,len(AIDS)).astype('int8')

In [10]:
dff = dff.set_index( ['aid','day'] )

In [11]:
dff.head()

Empty DataFrame
Columns: []
Index: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4)]

In [12]:
dff = dff.merge(tmp, left_index=True, right_index=True, how='left').fillna(0).reset_index()

In [13]:
dff.head()

,aid,day,item,user
0,375,7,0,0
1,375,8,0,0
2,375,9,0,0
3,375,10,0,0
4,375,11,0,0


In [14]:
item_features3 = dff.groupby('aid').agg({'item':'std','user':'std'}).fillna(-1)
item_features3.columns = ['item_day_std','user_day_std']

f32 = ['item_day_std','user_day_std']
for c in f32: item_features3[c] = item_features3[c].astype('float32')

In [15]:
del dff
gc.collect()

86

In [16]:
item_features3.head()

,item_day_std,user_day_std
aid,,
102299,1.627151,1.627151
223409,0.436436,0.436436
41691,0.560612,0.560612
132253,0.300793,0.300793
1617899,6.720261,6.720261


In [17]:
df = cudf.read_parquet('/raid/Kaggle/otto/parquet2/train.parquet')
df.ts = (df.ts//1000).astype('int32')

mn = df.ts.min()
#df['hour'] = df.ts % (60*60*24)
df['day'] = (df.ts-mn)  // (60*60*24) #% (60*60*24*7)
print( df.shape )
df.head()

(216716096, 5)


,session,aid,ts,type,day
0,0,1517085,1659304800,0,0
1,0,1563459,1659304904,0,0
2,0,1309446,1659367439,0,0
3,0,16246,1659367719,0,0
4,0,1781822,1659367871,0,0


In [18]:
df['mn'] = df.groupby('session').ts.transform('min')

In [19]:
df = df.loc[df.mn > mn + 60*60*24*3]
df.shape

(125864676, 6)

In [20]:
df['t'] = df.ts - df.mn

In [21]:
df = df.loc[df.t < 60*60*24*7]
df.shape

(92222573, 7)

In [22]:
item_features = df.groupby('aid').agg({'type':'mean','aid':'count','session':'nunique'})
item_features.columns = ['buy_ratio','count_item','count_user']

In [23]:
item_features.head()

,buy_ratio,count_item,count_user
aid,,,
0,0.000000,20,17
1,0.052632,19,17
2,0.000000,11,10
3,0.104082,1470,715
4,0.000000,64,51


In [24]:
item_features2 = df.groupby('aid').agg({'type':'std'}).fillna(-1)
item_features2.columns = ['buy_ratio_std']

f32 = ['buy_ratio_std']
for c in f32: item_features2[c] = item_features2[c].astype('float32')

In [25]:
item_features2.head()

,buy_ratio_std
aid,
1133385,0.000000
1180295,0.223456
434755,0.000000
371971,0.134840
1015389,0.374166


In [26]:
item_features['repeat'] = item_features.count_item / item_features.count_user

f32 = ['buy_ratio','repeat']
for c in f32: item_features[c] = item_features[c].astype('float32')

i32 = ['count_item','count_user']
for c in i32: item_features[c] = item_features[c].astype('int32')

In [27]:
for d in [item_features,item_features2,item_features3]:
    print( d.shape )

(1797312, 4)
(1797312, 1)
(1839558, 2)


In [28]:
item_features = cudf.concat([item_features,item_features2,item_features3],axis=1)

In [29]:
item_features.shape

(1848283, 7)

In [30]:
item_features.columns = [f'{c}_5' for c in item_features.columns]

In [31]:
item_features.head()

,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5
aid,,,,,,,
0,0.000000,20,17,1.176471,0.000000,<NA>,<NA>
1,0.052632,19,17,1.117647,0.229416,<NA>,<NA>
2,0.000000,11,10,1.100000,0.000000,<NA>,<NA>
3,0.104082,1470,715,2.055944,0.353024,<NA>,<NA>
4,0.000000,64,51,1.254902,0.000000,<NA>,<NA>


In [32]:
item_features.dtypes

buy_ratio_5        float32
count_item_5         int32
count_user_5         int32
repeat_5           float32
buy_ratio_std_5    float32
item_day_std_5     float32
user_day_std_5     float32
dtype: object

In [33]:
for k in range(7):
    n = item_features.iloc[:,k].isna().sum()
    print( item_features.columns[k], n)

buy_ratio_5 50971
count_item_5 50971
count_user_5 50971
repeat_5 50971
buy_ratio_std_5 50971
item_day_std_5 1848283
user_day_std_5 1848283


In [34]:
item_features = item_features.astype('float32')

In [35]:
item_features.to_parquet('../../data/item_user_features/item14b_LB.pqt')